In [1]:
import os
import numpy as np


In [2]:
main_dir = '/Users/praneeth/Downloads/Datasets/aclImdb'
train_dir = os.path.join(main_dir, 'train')

In [3]:
temp1 = '/Users/praneeth/Downloads/Datasets/aclImdb/train/neg'
temp = '/Users/praneeth/Downloads/Datasets/aclImdb/train/neg/0_3.txt'



In [4]:
f = open(temp, "r")
print(f.read())
f.close()

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.


In [5]:
labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            file_link = os.path.join(dir_name, fname)
            f = open(file_link, "r")
            texts.append(f.read())
            f.close()
        if label_type == 'neg':
            labels.append(0)
        else:
            labels.append(1)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [7]:
max_words = 10000

In [8]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [9]:
word_index = tokenizer.word_index
len(word_index)

import keras
max_length = 100
padded_docs = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, padding = 'post')
print(padded_docs)

[[  34   44 7576 ...   27  107 3348]
 [4311   44  298 ...    1 4517 5457]
 [ 382  635   14 ... 1906   15  620]
 ...
 [   3 3042  134 ... 1110  205 9755]
 [   8  202   15 ... 3713   14   70]
 [   4 3323  664 ...   82   93   36]]


In [10]:
labels = np.asarray(labels)
padded_docs.shape

(25000, 100)

In [11]:
word_index = tokenizer.word_index
len(word_index)
indices = np.arange(25000)

np.random.shuffle(indices)

shuffled_data = padded_docs[indices]
shuffled_labels = labels[indices]

In [12]:
training_samples = 100
validation_samples = 10000

x_train = shuffled_data[0:training_samples]
y_train = shuffled_labels[0:training_samples]

x_val = shuffled_data[training_samples:training_samples+validation_samples]
y_val = shuffled_labels[training_samples:training_samples+validation_samples]




In [13]:
from matplotlib import pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/praneeth/anaconda3/envs/tensorflow/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [14]:
glove_dir = '/Users/praneeth/Downloads/Self learning AI:ML/Chap_6_DLP/glove.6B/glove.6B.100d.txt'

In [15]:
f = open(glove_dir, "r")

embeddings_index = {}

for line in f:
    values = line.split()
    word = values[0]
    rep = np.asarray(values[1:], dtype = 'float32')
    embeddings_index[word] = rep



In [16]:
#About 4 lakh values are present in the embeddings_index dictionary
len(embeddings_index)

400001

In [17]:
embedding_dimension = 100
embeddings_matrix = np.zeros(shape = ( max_words,embedding_dimension  ))

for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector

        

In [18]:
embedding_dim = 100
max_words = 10000
max_length = 100

In [19]:
from keras.layers import Embedding




In [20]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_length))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model.layers[0].set_weights([embeddings_matrix])
model.layers[0].trainable = False

In [22]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 320,065
Non-trainable params: 1,000,000
_________________________________________________________________


In [23]:
len(word_index)

88582

In [24]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('/Users/praneeth/Downloads/Self learning AI:ML/Chap_6_DLP/pre_trained_glove_model.h5')

Train on 100 samples, validate on 10000 samples
Epoch 1/10
100/100 [==============================] - 3s 33ms/step - loss: 3.1115 - acc: 0.4600 - val_loss: 0.7219 - val_acc: 0.5296
Epoch 2/10
100/100 [==============================] - 1s 11ms/step - loss: 0.8421 - acc: 0.6300 - val_loss: 1.1526 - val_acc: 0.5039
Epoch 3/10
100/100 [==============================] - 1s 13ms/step - loss: 0.5823 - acc: 0.6500 - val_loss: 0.7096 - val_acc: 0.5611
Epoch 4/10
100/100 [==============================] - 2s 16ms/step - loss: 0.1629 - acc: 0.9700 - val_loss: 0.7823 - val_acc: 0.5443
Epoch 5/10
100/100 [==============================] - 1s 14ms/step - loss: 0.1151 - acc: 1.0000 - val_loss: 0.7525 - val_acc: 0.5535
Epoch 6/10
100/100 [==============================] - 1s 11ms/step - loss: 0.0487 - acc: 1.0000 - val_loss: 0.9379 - val_acc: 0.5269
Epoch 7/10
100/100 [==============================] - 1s 12ms/step - loss: 0.0384 - acc: 1.0000 - val_loss: 0.8079 - val_acc: 0.5423
Epoch 8/10
100/100 [=

In [25]:
history.history.keys()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])

In [50]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)




In [ ]:
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


